In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt

from img2vec import rgb2emb, get_resnet_model

### Check for GPU

In [ ]:
# Check if GPU is available
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Print GPU information
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        print("Name:", gpu.name, "  Type:", gpu.device_type)

    # Set memory growth to avoid using all GPU memory
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)

    print("GPU is available for TensorFlow!")
else:
    print("No GPU found. TensorFlow will use CPU.")

### Set random seed for reproducibility

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

### Define data paths

In [ ]:
data_dir = os.path.join('..', '..', 'data')
faces_dir = os.path.join(data_dir, 'faces')
train_csv = os.path.join(data_dir, 'train.csv')
val_csv = os.path.join(data_dir, 'val.csv')
test_csv = os.path.join(data_dir, 'test.csv')

### Load data

In [ ]:
train_df = pd.read_csv(train_csv)
val_df = pd.read_csv(val_csv)
test_df = pd.read_csv(test_csv)

### Print dataset information

In [ ]:
print(f"Training set size: {len(train_df)}")
print(f"Validation set size: {len(val_df)}")
print(f"Test set size: {len(test_df)}")

### Create image generators with data augmentation for training

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Only rescale for validation and test data (no augmentation)
val_test_datagen = ImageDataGenerator(rescale=1. / 255)

### Create a function to map age ranges to numerical labels

In [ ]:
def map_age_to_label(age_range):
    age_mapping = {
        '(0, 2)': 0,
        '(4, 6)': 1,
        '(8, 23)': 2,
        '(15, 20)': 3,
        '(25, 32)': 4,
        '(38, 43)': 5,
        '(48, 53)': 6,
        '(60, 100)': 7
    }
    return age_mapping[age_range]

### Create a function to map gender to numerical labels

In [ ]:
def map_gender_to_label(gender):
    gender_mapping = {
        'm': 0,  # male
        'f': 1,  # female
        'u': 2  # unknown
    }
    return gender_mapping[gender]

### Add numerical labels to dataframes

In [ ]:
train_df['age_label'] = train_df['age'].apply(map_age_to_label)
train_df['gender_label'] = train_df['gender'].apply(map_gender_to_label)

val_df['age_label'] = val_df['age'].apply(map_age_to_label)
val_df['gender_label'] = val_df['gender'].apply(map_gender_to_label)

test_df['age_label'] = test_df['age'].apply(map_age_to_label)
test_df['gender_label'] = test_df['gender'].apply(map_gender_to_label)

### Convert labels to one-hot encoding

In [ ]:
num_age_classes = 8
num_gender_classes = 3


def to_one_hot(label, num_classes):
    one_hot = np.zeros(num_classes)
    one_hot[label] = 1
    return one_hot

### Extract image paths from dataframes

In [ ]:
def get_image_paths(df):
    image_paths = []
    for _, row in df.iterrows():
        # Construct the image path from user_id and original_image
        # Make sure this path matches your actual folder structure
        img_path = os.path.join(data_dir, 'faces', row['user_id'],
                                f"coarse_tilt_aligned_face.{row['face_id']}.{row['original_image']}")
        image_paths.append(img_path)
    return image_paths

### Create custom data generator using the rgb2emb function

In [ ]:
class ImageFeatureGenerator:
    def __init__(self, dataframe, batch_size=32, shuffle=True):
        self.dataframe = dataframe
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indices = np.arange(len(dataframe))
        self.steps = len(dataframe) // batch_size
        
    def __len__(self):
        return self.steps
    
    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)
    
    def __iter__(self):
        self.on_epoch_end()
        self.current_batch = 0
        return self
    
    def __next__(self):
        if self.current_batch >= self.steps:
            raise StopIteration
        
        batch_indices = self.indices[self.current_batch * self.batch_size:(self.current_batch + 1) * self.batch_size]
        batch_df = self.dataframe.iloc[batch_indices]
        
        # Get image paths for this batch
        img_paths = get_image_paths(batch_df)
        
        # Extract features using rgb2emb function
        features = rgb2emb(img_paths, batch_size=self.batch_size)
        
        # Create one-hot encoded labels
        age_labels = np.array([to_one_hot(label, num_age_classes) for label in batch_df['age_label']])
        gender_labels = np.array([to_one_hot(label, num_gender_classes) for label in batch_df['gender_label']])
        
        self.current_batch += 1
        return features, [age_labels, gender_labels]


### Create generators

In [ ]:
batch_size = 32
train_generator = ImageFeatureGenerator(train_df, batch_size=batch_size)
val_generator = ImageFeatureGenerator(val_df, batch_size=batch_size)
test_generator = ImageFeatureGenerator(test_df, batch_size=batch_size)

### Build a model that takes pre-extracted features

In [ ]:
def build_model_from_features(feature_dim=2048):
    # Input is the pre-extracted features
    inputs = layers.Input(shape=(feature_dim,))
    
    # Common dense layers
    x = layers.Dense(512, activation='relu')(inputs)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    
    # Age classification branch
    age_branch = layers.Dense(128, activation='relu')(x)
    age_branch = layers.Dropout(0.3)(age_branch)
    age_output = layers.Dense(num_age_classes, activation='softmax', name='age_output')(age_branch)
    
    # Gender classification branch
    gender_branch = layers.Dense(64, activation='relu')(x)
    gender_branch = layers.Dropout(0.3)(gender_branch)
    gender_output = layers.Dense(num_gender_classes, activation='softmax', name='gender_output')(gender_branch)
    
    # Create the model
    model = models.Model(inputs=inputs, outputs=[age_output, gender_output])
    
    # Compile the model
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss={
            'age_output': 'categorical_crossentropy',
            'gender_output': 'categorical_crossentropy'
        },
        metrics={
            'age_output': 'accuracy',
            'gender_output': 'accuracy'
        }
    )
    
    return model


### Create the model

In [ ]:
feature_dim = 2048  # Output dimension of the RGB to embedding function
model = build_model_from_features(feature_dim)

# Display model summary
model.summary()

### Define callback for GPU monitoring during training

In [ ]:
class GPUUtilizationCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        try:
            import subprocess
            result = subprocess.run(['nvidia-smi'], stdout=subprocess.PIPE)
            print(result.stdout.decode('utf-8'))
        except Exception as e:
            print(f"Could not run nvidia-smi: {e}")


### Define callbacks

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

checkpoint = tf.keras.callbacks.ModelCheckpoint(
    'best_resnet_model.h5',
    save_best_only=True,
    monitor='val_loss'
)

gpu_callback = GPUUtilizationCallback()

### Convert generators to tf.data.Dataset for more efficient training

In [ ]:
def generator_to_dataset(generator, batch_size):
    def gen():
        for features, labels in generator:
            yield features, labels
    
    # Define the output signature
    output_signature = (
        tf.TensorSpec(shape=(None, feature_dim), dtype=tf.float32),
        (
            tf.TensorSpec(shape=(None, num_age_classes), dtype=tf.float32),
            tf.TensorSpec(shape=(None, num_gender_classes), dtype=tf.float32)
        )
    )
    
    dataset = tf.data.Dataset.from_generator(
        gen,
        output_signature=output_signature
    )
    
    return dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

# Create tf.data.Dataset objects
train_dataset = generator_to_dataset(train_generator, batch_size)
val_dataset = generator_to_dataset(val_generator, batch_size)
test_dataset = generator_to_dataset(test_generator, batch_size)

### Train the model

In [ ]:
epochs = 20
history = model.fit(
    train_dataset,
    epochs=epochs,
    validation_data=val_dataset,
    callbacks=[early_stopping, checkpoint, gpu_callback]
)

### Plot training history

In [ ]:
def plot_history(history):
    # Plot loss
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    # Plot accuracy
    plt.subplot(1, 2, 2)
    plt.plot(history.history['age_output_accuracy'], label='Age Accuracy')
    plt.plot(history.history['gender_output_accuracy'], label='Gender Accuracy')
    plt.plot(history.history['val_age_output_accuracy'], label='Val Age Accuracy')
    plt.plot(history.history['val_gender_output_accuracy'], label='Val Gender Accuracy')
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig('training_history.png')
    plt.show()

plot_history(history)

### Evaluate the model on test data

In [ ]:
print("Evaluating model on test data...")
results = model.evaluate(test_dataset, verbose=1)
print("Test results:", results)

### Make predictions

In [ ]:
print("Generating predictions...")
test_predictions = model.predict(test_dataset)
age_predictions = np.argmax(test_predictions[0], axis=1)
gender_predictions = np.argmax(test_predictions[1], axis=1)


### Print classification reports

In [ ]:
# Get true labels
test_age_true = np.array(test_df['age_label'])
test_gender_true = np.array(test_df['gender_label'])

print("\nAge Classification Report:")
print(classification_report(test_age_true, age_predictions))

print("\nGender Classification Report:")
print(classification_report(test_gender_true, gender_predictions))

### Create confusion matrices

In [ ]:
age_cm = confusion_matrix(test_age_true, age_predictions)
gender_cm = confusion_matrix(test_gender_true, gender_predictions)

# Plot confusion matrices
plt.figure(figsize=(16, 6))

plt.subplot(1, 2, 1)
plt.imshow(age_cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Age Confusion Matrix')
plt.colorbar()
age_classes = ['0-2', '4-6', '8-23', '15-20', '25-32', '38-43', '48-53', '60+']
plt.xticks(np.arange(len(age_classes)), age_classes, rotation=45)
plt.yticks(np.arange(len(age_classes)), age_classes)
plt.ylabel('True Age')
plt.xlabel('Predicted Age')

plt.subplot(1, 2, 2)
plt.imshow(gender_cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Gender Confusion Matrix')
plt.colorbar()
gender_classes = ['Male', 'Female', 'Unknown']
plt.xticks(np.arange(len(gender_classes)), gender_classes)
plt.yticks(np.arange(len(gender_classes)), gender_classes)
plt.ylabel('True Gender')
plt.xlabel('Predicted Gender')

plt.tight_layout()
plt.savefig('confusion_matrices.png')
plt.show()

### Save the final model

In [ ]:
model.save('final_age_gender_model.h5')